In [423]:
data = """2333133121414131402"""

In [408]:
with open("AoC_D9_data.txt", "r") as f:
    data = f.read()

In [424]:
data = list(data)

In [425]:
def disk_map_to_blocks(disk_map):
    blocks_map = []
    for n, i in enumerate(list(disk_map)):
        if n%2==0:
            blocks_map.extend([str(int(n/2))]*int(i))
        else:
            blocks_map.extend(["."]*int(i))
    return blocks_map

In [426]:
def compactify_blocks(blocks_map):
    last_file_block = -1
    while '.' in "".join(blocks_map).rstrip('.'):
        ind_space = blocks_map.index('.')
        for n, v in enumerate(blocks_map[::-1]):
            if v != '.': break
        ind_last_file_block = len(blocks_map) - 1 - n        
        if last_file_block != blocks_map[ind_last_file_block]:
            if int(blocks_map[ind_last_file_block])%100==0:
                print(blocks_map[ind_last_file_block])
        last_file_block = blocks_map[ind_last_file_block]
        blocks_map.pop(ind_last_file_block)
        blocks_map.pop(blocks_map.index('.'))
        blocks_map.insert(ind_space, last_file_block)
        blocks_map = blocks_map + ['.']
        # print("".join(blocks_map))
    return blocks_map

In [427]:
def calculate_checksum(blocks_map):
    cs = 0
    for i, v in enumerate(blocks_map):
        if v == ".":
            continue
        cs += int(v)*i
    return cs

In [428]:
blocks_map =  disk_map_to_blocks(data)
blocks_map = compactify_blocks(blocks_map)
calculate_checksum(blocks_map)

1928

In [429]:
# part 2 
import pandas as pd
from tqdm import tqdm

In [435]:
def calculate_free_map(blocks_map):
    s = pd.to_numeric(pd.Series(blocks_map).replace({".":"-1"}))
    s = s[s==-1].reset_index()
    s['index_cum'] = (s['index'].diff()!=1).cumsum()-1
    free_map = s.groupby('index_cum').agg({'index':"first", 0:"count"})
    free_map.columns = ["start_index", "size"]
    free_map.index.name = "free_index"
    return free_map

def group_file_map(blocks_map):
    s = pd.to_numeric(pd.Series(blocks_map).replace({".":"-1"}))
    s = s[s!=-1].reset_index()
    s['index_cum'] = (s[0].diff()!=0).cumsum()-1
    file_map = s.groupby('index_cum').agg({'index':"first", 0:"count"})
    file_map.columns = ["start_index", "size"]
    file_map.index.name = "file_id"
    return file_map

def compactify_all_blocks(blocks_map, file_map, free_map):
    max_file_id = pd.to_numeric(pd.Series(blocks_map).replace({".":"-1"})).max()
    for fid in tqdm(range(max_file_id, -1, -1)):
        file_size = file_map.loc[fid, "size"]
        file_index = file_map.loc[fid, "start_index"]
        available_gaps = free_map.loc[(free_map['start_index']<file_index) & (free_map['size']>=file_size), :]
        if len(available_gaps)>0:
            new_index = available_gaps.iloc[0, :]["start_index"]
            for i in range(file_size):
                blocks_map[file_index+i] = '.'
                blocks_map[new_index+i] = str(fid)
        free_map = calculate_free_map(blocks_map)
    return blocks_map
    

In [436]:
blocks_map =  disk_map_to_blocks(data)
free_map = calculate_free_map(blocks_map)
file_map = group_file_map(blocks_map)
blocks_map = compactify_all_blocks(blocks_map, file_map, free_map)
calculate_checksum(blocks_map)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 275.19it/s]


2858